In [1]:
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 9999

In [2]:
train =  pd.read_excel("Data_Train.xlsx")
test = pd.read_excel("Data_Test.xlsx")
sample = pd.read_excel("Sample_Submission.xlsx")
train.shape, test.shape, sample.shape

((11094, 9), (2774, 8), (2774, 1))

In [3]:
train.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",₹100,₹50,3.5,11,4,30 minutes
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",₹150,₹50,3.6,99,30,65 minutes
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",₹250,₹99,3.7,176,95,30 minutes
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",₹200,₹99,3.2,521,235,65 minutes


In [4]:
test.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews
0,ID_2842,"Mico Layout, Stage 2, BTM Layout,Bangalore","North Indian, Chinese, Assamese",₹350,₹50,4.2,361,225
1,ID_730,"Mico Layout, Stage 2, BTM Layout,Bangalore","Biryani, Kebab",₹100,₹50,NEW,-,-
2,ID_4620,"Sector 1, Noida",Fast Food,₹100,₹50,3.6,36,16
3,ID_5470,"Babarpur, New Delhi, Delhi","Mithai, North Indian, Chinese, Fast Food, Sout...",₹200,₹50,3.6,66,33
4,ID_3249,"Sector 1, Noida","Chinese, Fast Food",₹150,₹50,2.9,38,14


In [5]:
sample.head()

,Delivery_Time
0,120 minutes
1,20 minutes
2,20 minutes
3,65 minutes
4,45 minutes


In [6]:
test['Delivery_Time'] = '0 minutes'
train['train_test'] = 'train'
test['train_test'] = 'test'

In [7]:
total_df = pd.concat([train,test],axis=0,ignore_index=True,sort=False)
total_df.shape,train.shape,test.shape

((13868, 10), (11094, 10), (2774, 10))

In [8]:
total_df.tail()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,train_test
13863,ID_6342,Delhi University-GTB Nagar,"Fast Food, Chinese",₹100,₹50,NEW,-,-,0 minutes,test
13864,ID_8495,"Mico Layout, Stage 2, BTM Layout,Bangalore","Continental, North Indian, Desserts, Beverages",₹250,₹50,3.1,5,1,0 minutes,test
13865,ID_7122,"Noorkhan Bazaar, Malakpet, Hyderabad","Andhra, South Indian",₹150,₹50,3.0,16,1,0 minutes,test
13866,ID_2475,"D-Block, Sector 63, Noida",Bakery,₹100,₹99,3.0,7,2,0 minutes,test
13867,ID_1595,"Dockyard Road, Mumbai CST Area","Italian, Street Food, Fast Food",₹150,₹50,3.6,99,30,0 minutes,test


In [9]:
total_df['Rating'].replace("-",0,inplace= True)
total_df['Rating'].replace("Opening Soon",99,inplace= True)
total_df['Rating'].replace("NEW",999,inplace= True)
total_df['Rating'].replace("Temporarily Closed",9999,inplace= True)
total_df['Rating'] = pd.to_numeric(total_df['Rating'])

total_df['Votes'].replace("-",0,inplace= True)
total_df['Reviews'].replace("-",0,inplace= True)

total_df['Average_Cost'] = total_df['Average_Cost'].map(lambda x: x.lstrip('₹').rstrip('aAbBcC'))
total_df['Minimum_Order'] = total_df['Minimum_Order'].map(lambda x: x.lstrip('₹').rstrip('aAbBcC'))
total_df['Average_Cost'] = total_df['Average_Cost'].str.replace(',', '')

total_df['Average_Cost'].replace("for",100,inplace= True)

total_df['Minimum_Order'] = total_df['Minimum_Order'].astype(float)
total_df['Average_Cost'] = total_df['Average_Cost'].astype(float)
total_df['Votes'] = total_df['Votes'].astype(float)
total_df['Reviews'] = total_df['Reviews'].astype(float)

total_df.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,train_test
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",200.0,50.0,3.5,12.0,4.0,30 minutes,train
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",100.0,50.0,3.5,11.0,4.0,30 minutes,train
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",150.0,50.0,3.6,99.0,30.0,65 minutes,train
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",250.0,99.0,3.7,176.0,95.0,30 minutes,train
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",200.0,99.0,3.2,521.0,235.0,65 minutes,train


In [10]:
total_df['Ratings'] = 'Ratings'
total_df['Ratings'] = np.where(total_df['Rating'] >= 9999 , 'Temporarily Close',
                               np.where(total_df['Rating'] >= 999 , 'NEW',
                                        np.where(total_df['Rating'] >= 99 , 'Opening Soon',
                                                 np.where(total_df['Rating'] >= 4 , 'Above 4', 
                                                          np.where(total_df['Rating'] >= 3.5 , 'Above 3.5',
                                                                   np.where(total_df['Rating'] >= 3 , 'Above 3',
                                                                            np.where(total_df['Rating'] >= 2 , 'Above 2',
                                                                                     np.where(total_df['Rating'] >= 1 , 'Above 1',
                                                                                              np.where(total_df['Rating'] > 0 , 'Above 0', 'Not Rated')))))))))
total_df.sample(10)

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,train_test,Ratings
6965,ID_6168,"Delhi Administration Flats, Timarpur",Paan,100.0,50.0,0.0,0.0,0.0,30 minutes,train,Not Rated
45,ID_6868,Majestic,"North Indian, Street Food",200.0,50.0,3.8,37.0,25.0,30 minutes,train,Above 3.5
10465,ID_1424,"Delhi High Court, India Gate","North Indian, Continental, Chinese, Fast Food",800.0,99.0,4.0,1852.0,1130.0,45 minutes,train,Above 4
12530,ID_7471,Mumbai Central,"Mithai, Street Food",200.0,50.0,4.1,502.0,228.0,0 minutes,test,Above 4
11068,ID_8063,Delhi University-GTB Nagar,Fast Food,100.0,50.0,2.8,8.0,3.0,30 minutes,train,Above 2
12437,ID_7009,"FTI College, Law College Road, Pune",Chinese,150.0,50.0,3.6,39.0,3.0,0 minutes,test,Above 3.5
12925,ID_5423,"Rmz Centennial, I Gate, Whitefield","Desserts, Ice Cream",200.0,50.0,0.0,0.0,0.0,0 minutes,test,Not Rated
11571,ID_7276,"Sector 14, Noida","North Indian, Mughlai",300.0,50.0,4.4,308.0,175.0,0 minutes,test,Above 4
8289,ID_4702,"Laxman Vihar Industrial Area, Sector 3A, Gurgoan",North Indian,100.0,50.0,3.8,106.0,17.0,30 minutes,train,Above 3.5
3895,ID_1431,"Sector 3, Marathalli","Beverages, Fast Food",150.0,50.0,3.2,4.0,3.0,30 minutes,train,Above 3


In [11]:
total_df['Ratings'].value_counts()

Above 3.5            5113
Above 3              3269
Above 4              2351
Not Rated            1496
NEW                   958
Above 2               665
Opening Soon           14
Temporarily Close       2
Name: Ratings, dtype: int64

In [12]:
total_df.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,train_test,Ratings
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",200.0,50.0,3.5,12.0,4.0,30 minutes,train,Above 3.5
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",100.0,50.0,3.5,11.0,4.0,30 minutes,train,Above 3.5
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",150.0,50.0,3.6,99.0,30.0,65 minutes,train,Above 3.5
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",250.0,99.0,3.7,176.0,95.0,30 minutes,train,Above 3.5
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",200.0,99.0,3.2,521.0,235.0,65 minutes,train,Above 3


In [13]:
import re
total_df['Cuisines'] = [re.sub(' ','',i)for i in total_df.Cuisines]
total_df['Cuisines'] = total_df['Cuisines'].apply(lambda x: x.replace(',',' '))
total_df['no_cuisines'] = total_df['Cuisines'].apply(lambda x: len(re.findall(r'\w+',x)))
total_df['no_cuisines'] = total_df['no_cuisines'].astype(int)
total_df.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,train_test,Ratings,no_cuisines
0,ID_6321,"FTI College, Law College Road, Pune",FastFood Rolls Burger Salad Wraps,200.0,50.0,3.5,12.0,4.0,30 minutes,train,Above 3.5,5
1,ID_2882,"Sector 3, Marathalli",IceCream Desserts,100.0,50.0,3.5,11.0,4.0,30 minutes,train,Above 3.5,2
2,ID_1595,Mumbai Central,Italian StreetFood FastFood,150.0,50.0,3.6,99.0,30.0,65 minutes,train,Above 3.5,3
3,ID_5929,"Sector 1, Noida",Mughlai NorthIndian Chinese,250.0,99.0,3.7,176.0,95.0,30 minutes,train,Above 3.5,3
4,ID_6123,"Rmz Centennial, I Gate, Whitefield",Cafe Beverages,200.0,99.0,3.2,521.0,235.0,65 minutes,train,Above 3,2


In [14]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(strip_accents='unicode',ngram_range=(1,2),decode_error='ignore')

tdm = cv.fit_transform(total_df.Cuisines)

Mat = tdm.todense()

mat_df =  pd.DataFrame(Mat)

mat_df.shape

(13868, 1290)

In [15]:
cols = mat_df.columns
new_cols = []
for i in range(len(cols)):
    new_cols.append("Cuisine_"+str(i))
    
mat_df.columns = new_cols
mat_df = mat_df.astype('category')

In [16]:
mat_df.head()

,Cuisine_0,Cuisine_1,Cuisine_2,Cuisine_3,Cuisine_4,Cuisine_5,Cuisine_6,Cuisine_7,Cuisine_8,Cuisine_9,Cuisine_10,Cuisine_11,Cuisine_12,Cuisine_13,Cuisine_14,Cuisine_15,Cuisine_16,Cuisine_17,Cuisine_18,Cuisine_19,Cuisine_20,Cuisine_21,Cuisine_22,Cuisine_23,Cuisine_24,Cuisine_25,Cuisine_26,Cuisine_27,Cuisine_28,Cuisine_29,Cuisine_30,Cuisine_31,Cuisine_32,Cuisine_33,Cuisine_34,Cuisine_35,Cuisine_36,Cuisine_37,Cuisine_38,Cuisine_39,Cuisine_40,Cuisine_41,Cuisine_42,Cuisine_43,Cuisine_44,Cuisine_45,Cuisine_46,Cuisine_47,Cuisine_48,Cuisine_49,Cuisine_50,Cuisine_51,Cuisine_52,Cuisine_53,Cuisine_54,Cuisine_55,Cuisine_56,Cuisine_57,Cuisine_58,Cuisine_59,Cuisine_60,Cuisine_61,Cuisine_62,Cuisine_63,Cuisine_64,Cuisine_65,Cuisine_66,Cuisine_67,Cuisine_68,Cuisine_69,Cuisine_70,Cuisine_71,Cuisine_72,Cuisine_73,Cuisine_74,Cuisine_75,Cuisine_76,Cuisine_77,Cuisine_78,Cuisine_79,Cuisine_80,Cuisine_81,Cuisine_82,Cuisine_83,Cuisine_84,Cuisine_85,Cuisine_86,Cuisine_87,Cuisine_88,Cuisine_89,Cuisine_90,Cuisine_91,Cuisine_92,Cuisine_93,Cuisine_94,Cuisine_95,Cuisine_96,Cuisine_97,Cuisine_98,Cuisine_99,Cuisine_100,Cuisine_101,Cuisine_102,Cuisine_103,Cuisine_104,Cuisine_105,Cuisine_106,Cuisine_107,Cuisine_108,Cuisine_109,Cuisine_110,Cuisine_111,Cuisine_112,Cuisine_113,Cuisine_114,Cuisine_115,Cuisine_116,Cuisine_117,Cuisine_118,Cuisine_119,Cuisine_120,Cuisine_121,Cuisine_122,Cuisine_123,Cuisine_124,Cuisine_125,Cuisine_126,Cuisine_127,Cuisine_128,Cuisine_129,Cuisine_130,Cuisine_131,Cuisine_132,Cuisine_133,Cuisine_134,Cuisine_135,Cuisine_136,Cuisine_137,Cuisine_138,Cuisine_139,Cuisine_140,Cuisine_141,Cuisine_142,Cuisine_143,Cuisine_144,Cuisine_145,Cuisine_146,Cuisine_147,Cuisine_148,Cuisine_149,Cuisine_150,Cuisine_151,Cuisine_152,Cuisine_153,Cuisine_154,Cuisine_155,Cuisine_156,Cuisine_157,Cuisine_158,Cuisine_159,Cuisine_160,Cuisine_161,Cuisine_162,Cuisine_163,Cuisine_164,Cuisine_165,Cuisine_166,Cuisine_167,Cuisine_168,Cuisine_169,Cuisine_170,Cuisine_171,Cuisine_172,Cuisine_173,Cuisine_174,Cuisine_175,Cuisine_176,Cuisine_177,Cuisine_178,Cuisine_179,Cuisine_180,Cuisine_181,Cuisine_182,Cuisine_183,Cuisine_184,Cuisine_185,Cuisine_186,Cuisine_187,Cuisine_188,Cuisine_189,Cuisine_190,Cuisine_191,Cuisine_192,Cuisine_193,Cuisine_194,Cuisine_195,Cuisine_196,Cuisine_197,Cuisine_198,Cuisine_199,Cuisine_200,Cuisine_201,Cuisine_202,Cuisine_203,Cuisine_204,Cuisine_205,Cuisine_206,Cuisine_207,Cuisine_208,Cuisine_209,Cuisine_210,Cuisine_211,Cuisine_212,Cuisine_213,Cuisine_214,Cuisine_215,Cuisine_216,Cuisine_217,Cuisine_218,Cuisine_219,Cuisine_220,Cuisine_221,Cuisine_222,Cuisine_223,Cuisine_224,Cuisine_225,Cuisine_226,Cuisine_227,Cuisine_228,Cuisine_229,Cuisine_230,Cuisine_231,Cuisine_232,Cuisine_233,Cuisine_234,Cuisine_235,Cuisine_236,Cuisine_237,Cuisine_238,Cuisine_239,Cuisine_240,Cuisine_241,Cuisine_242,Cuisine_243,Cuisine_244,Cuisine_245,Cuisine_246,Cuisine_247,Cuisine_248,Cuisine_249,Cuisine_250,Cuisine_251,Cuisine_252,Cuisine_253,Cuisine_254,Cuisine_255,Cuisine_256,Cuisine_257,Cuisine_258,Cuisine_259,Cuisine_260,Cuisine_261,Cuisine_262,Cuisine_263,Cuisine_264,Cuisine_265,Cuisine_266,Cuisine_267,Cuisine_268,Cuisine_269,Cuisine_270,Cuisine_271,Cuisine_272,Cuisine_273,Cuisine_274,Cuisine_275,Cuisine_276,Cuisine_277,Cuisine_278,Cuisine_279,Cuisine_280,Cuisine_281,Cuisine_282,Cuisine_283,Cuisine_284,Cuisine_285,Cuisine_286,Cuisine_287,Cuisine_288,Cuisine_289,Cuisine_290,Cuisine_291,Cuisine_292,Cuisine_293,Cuisine_294,Cuisine_295,Cuisine_296,Cuisine_297,Cuisine_298,Cuisine_299,Cuisine_300,Cuisine_301,Cuisine_302,Cuisine_303,Cuisine_304,Cuisine_305,Cuisine_306,Cuisine_307,Cuisine_308,Cuisine_309,Cuisine_310,Cuisine_311,Cuisine_312,Cuisine_313,Cuisine_314,Cuisine_315,Cuisine_316,Cuisine_317,Cuisine_318,Cuisine_319,Cuisine_320,Cuisine_321,Cuisine_322,Cuisine_323,Cuisine_324,Cuisine_325,Cuisine_326,Cuisine_327,Cuisine_328,Cuisine_329,Cuisine_330,Cuisine_331,Cuisine_332,Cuisine_333,Cuisine_334,Cuisine_335,Cuisine_336,Cuisine_337,Cuisine_338,Cuisine_339,Cuisine_340,Cuisine_341,Cuisi

In [17]:
total_df = pd.concat([total_df,mat_df],axis=1,sort=False)
total_df.shape

(13868, 1302)

In [18]:
total_df.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,train_test,Ratings,no_cuisines,Cuisine_0,Cuisine_1,Cuisine_2,Cuisine_3,Cuisine_4,Cuisine_5,Cuisine_6,Cuisine_7,Cuisine_8,Cuisine_9,Cuisine_10,Cuisine_11,Cuisine_12,Cuisine_13,Cuisine_14,Cuisine_15,Cuisine_16,Cuisine_17,Cuisine_18,Cuisine_19,Cuisine_20,Cuisine_21,Cuisine_22,Cuisine_23,Cuisine_24,Cuisine_25,Cuisine_26,Cuisine_27,Cuisine_28,Cuisine_29,Cuisine_30,Cuisine_31,Cuisine_32,Cuisine_33,Cuisine_34,Cuisine_35,Cuisine_36,Cuisine_37,Cuisine_38,Cuisine_39,Cuisine_40,Cuisine_41,Cuisine_42,Cuisine_43,Cuisine_44,Cuisine_45,Cuisine_46,Cuisine_47,Cuisine_48,Cuisine_49,Cuisine_50,Cuisine_51,Cuisine_52,Cuisine_53,Cuisine_54,Cuisine_55,Cuisine_56,Cuisine_57,Cuisine_58,Cuisine_59,Cuisine_60,Cuisine_61,Cuisine_62,Cuisine_63,Cuisine_64,Cuisine_65,Cuisine_66,Cuisine_67,Cuisine_68,Cuisine_69,Cuisine_70,Cuisine_71,Cuisine_72,Cuisine_73,Cuisine_74,Cuisine_75,Cuisine_76,Cuisine_77,Cuisine_78,Cuisine_79,Cuisine_80,Cuisine_81,Cuisine_82,Cuisine_83,Cuisine_84,Cuisine_85,Cuisine_86,Cuisine_87,Cuisine_88,Cuisine_89,Cuisine_90,Cuisine_91,Cuisine_92,Cuisine_93,Cuisine_94,Cuisine_95,Cuisine_96,Cuisine_97,Cuisine_98,Cuisine_99,Cuisine_100,Cuisine_101,Cuisine_102,Cuisine_103,Cuisine_104,Cuisine_105,Cuisine_106,Cuisine_107,Cuisine_108,Cuisine_109,Cuisine_110,Cuisine_111,Cuisine_112,Cuisine_113,Cuisine_114,Cuisine_115,Cuisine_116,Cuisine_117,Cuisine_118,Cuisine_119,Cuisine_120,Cuisine_121,Cuisine_122,Cuisine_123,Cuisine_124,Cuisine_125,Cuisine_126,Cuisine_127,Cuisine_128,Cuisine_129,Cuisine_130,Cuisine_131,Cuisine_132,Cuisine_133,Cuisine_134,Cuisine_135,Cuisine_136,Cuisine_137,Cuisine_138,Cuisine_139,Cuisine_140,Cuisine_141,Cuisine_142,Cuisine_143,Cuisine_144,Cuisine_145,Cuisine_146,Cuisine_147,Cuisine_148,Cuisine_149,Cuisine_150,Cuisine_151,Cuisine_152,Cuisine_153,Cuisine_154,Cuisine_155,Cuisine_156,Cuisine_157,Cuisine_158,Cuisine_159,Cuisine_160,Cuisine_161,Cuisine_162,Cuisine_163,Cuisine_164,Cuisine_165,Cuisine_166,Cuisine_167,Cuisine_168,Cuisine_169,Cuisine_170,Cuisine_171,Cuisine_172,Cuisine_173,Cuisine_174,Cuisine_175,Cuisine_176,Cuisine_177,Cuisine_178,Cuisine_179,Cuisine_180,Cuisine_181,Cuisine_182,Cuisine_183,Cuisine_184,Cuisine_185,Cuisine_186,Cuisine_187,Cuisine_188,Cuisine_189,Cuisine_190,Cuisine_191,Cuisine_192,Cuisine_193,Cuisine_194,Cuisine_195,Cuisine_196,Cuisine_197,Cuisine_198,Cuisine_199,Cuisine_200,Cuisine_201,Cuisine_202,Cuisine_203,Cuisine_204,Cuisine_205,Cuisine_206,Cuisine_207,Cuisine_208,Cuisine_209,Cuisine_210,Cuisine_211,Cuisine_212,Cuisine_213,Cuisine_214,Cuisine_215,Cuisine_216,Cuisine_217,Cuisine_218,Cuisine_219,Cuisine_220,Cuisine_221,Cuisine_222,Cuisine_223,Cuisine_224,Cuisine_225,Cuisine_226,Cuisine_227,Cuisine_228,Cuisine_229,Cuisine_230,Cuisine_231,Cuisine_232,Cuisine_233,Cuisine_234,Cuisine_235,Cuisine_236,Cuisine_237,Cuisine_238,Cuisine_239,Cuisine_240,Cuisine_241,Cuisine_242,Cuisine_243,Cuisine_244,Cuisine_245,Cuisine_246,Cuisine_247,Cuisine_248,Cuisine_249,Cuisine_250,Cuisine_251,Cuisine_252,Cuisine_253,Cuisine_254,Cuisine_255,Cuisine_256,Cuisine_257,Cuisine_258,Cuisine_259,Cuisine_260,Cuisine_261,Cuisine_262,Cuisine_263,Cuisine_264,Cuisine_265,Cuisine_266,Cuisine_267,Cuisine_268,Cuisine_269,Cuisine_270,Cuisine_271,Cuisine_272,Cuisine_273,Cuisine_274,Cuisine_275,Cuisine_276,Cuisine_277,Cuisine_278,Cuisine_279,Cuisine_280,Cuisine_281,Cuisine_282,Cuisine_283,Cuisine_284,Cuisine_285,Cuisine_286,Cuisine_287,Cuisine_288,Cuisine_289,Cuisine_290,Cuisine_291,Cuisine_292,Cuisine_293,Cuisine_294,Cuisine_295,Cuisine_296,Cuisine_297,Cuisine_298,Cuisine_299,Cuisine_300,Cuisine_301,Cuisine_302,Cuisine_303,Cuisine_304,Cuisine_305,Cuisine_306,Cuisine_307,Cuisine_308,Cuisine_309,Cuisine_310,Cuisine_311,Cuisine_312,Cuisine_313,Cuisine_314,Cuisine_315,Cuisine_316,Cuisine_317,Cuisine_318,Cuisine_319,Cuisine_320,Cuisine_321,Cuisine_322,Cuisine_323,Cuisine_324,Cuisine_325,Cuisine_326,Cuisine_327,Cuisine_328,Cuisine_329,Cuisine_330,Cuisine_331,Cui

In [19]:
cat_cols = ['Location','Ratings']

In [20]:
#using label encoder we are labelling all the categorical features similar to dummification(one hot encoding)
from sklearn import preprocessing

for col in cat_cols:
    print(col)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(total_df[col].values.astype('str')))
    total_df[col] = lbl.transform(list(total_df[col].values.astype('str')))
    total_df[col] = total_df[col].astype('category')

Location
Ratings


In [21]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(total_df['Delivery_Time'])
total_df['Delivery_Time'] = le.transform(total_df['Delivery_Time'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

{'0 minutes': 0, '10 minutes': 1, '120 minutes': 2, '20 minutes': 3, '30 minutes': 4, '45 minutes': 5, '65 minutes': 6, '80 minutes': 7}


In [22]:
total_df.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,train_test,Ratings,no_cuisines,Cuisine_0,Cuisine_1,Cuisine_2,Cuisine_3,Cuisine_4,Cuisine_5,Cuisine_6,Cuisine_7,Cuisine_8,Cuisine_9,Cuisine_10,Cuisine_11,Cuisine_12,Cuisine_13,Cuisine_14,Cuisine_15,Cuisine_16,Cuisine_17,Cuisine_18,Cuisine_19,Cuisine_20,Cuisine_21,Cuisine_22,Cuisine_23,Cuisine_24,Cuisine_25,Cuisine_26,Cuisine_27,Cuisine_28,Cuisine_29,Cuisine_30,Cuisine_31,Cuisine_32,Cuisine_33,Cuisine_34,Cuisine_35,Cuisine_36,Cuisine_37,Cuisine_38,Cuisine_39,Cuisine_40,Cuisine_41,Cuisine_42,Cuisine_43,Cuisine_44,Cuisine_45,Cuisine_46,Cuisine_47,Cuisine_48,Cuisine_49,Cuisine_50,Cuisine_51,Cuisine_52,Cuisine_53,Cuisine_54,Cuisine_55,Cuisine_56,Cuisine_57,Cuisine_58,Cuisine_59,Cuisine_60,Cuisine_61,Cuisine_62,Cuisine_63,Cuisine_64,Cuisine_65,Cuisine_66,Cuisine_67,Cuisine_68,Cuisine_69,Cuisine_70,Cuisine_71,Cuisine_72,Cuisine_73,Cuisine_74,Cuisine_75,Cuisine_76,Cuisine_77,Cuisine_78,Cuisine_79,Cuisine_80,Cuisine_81,Cuisine_82,Cuisine_83,Cuisine_84,Cuisine_85,Cuisine_86,Cuisine_87,Cuisine_88,Cuisine_89,Cuisine_90,Cuisine_91,Cuisine_92,Cuisine_93,Cuisine_94,Cuisine_95,Cuisine_96,Cuisine_97,Cuisine_98,Cuisine_99,Cuisine_100,Cuisine_101,Cuisine_102,Cuisine_103,Cuisine_104,Cuisine_105,Cuisine_106,Cuisine_107,Cuisine_108,Cuisine_109,Cuisine_110,Cuisine_111,Cuisine_112,Cuisine_113,Cuisine_114,Cuisine_115,Cuisine_116,Cuisine_117,Cuisine_118,Cuisine_119,Cuisine_120,Cuisine_121,Cuisine_122,Cuisine_123,Cuisine_124,Cuisine_125,Cuisine_126,Cuisine_127,Cuisine_128,Cuisine_129,Cuisine_130,Cuisine_131,Cuisine_132,Cuisine_133,Cuisine_134,Cuisine_135,Cuisine_136,Cuisine_137,Cuisine_138,Cuisine_139,Cuisine_140,Cuisine_141,Cuisine_142,Cuisine_143,Cuisine_144,Cuisine_145,Cuisine_146,Cuisine_147,Cuisine_148,Cuisine_149,Cuisine_150,Cuisine_151,Cuisine_152,Cuisine_153,Cuisine_154,Cuisine_155,Cuisine_156,Cuisine_157,Cuisine_158,Cuisine_159,Cuisine_160,Cuisine_161,Cuisine_162,Cuisine_163,Cuisine_164,Cuisine_165,Cuisine_166,Cuisine_167,Cuisine_168,Cuisine_169,Cuisine_170,Cuisine_171,Cuisine_172,Cuisine_173,Cuisine_174,Cuisine_175,Cuisine_176,Cuisine_177,Cuisine_178,Cuisine_179,Cuisine_180,Cuisine_181,Cuisine_182,Cuisine_183,Cuisine_184,Cuisine_185,Cuisine_186,Cuisine_187,Cuisine_188,Cuisine_189,Cuisine_190,Cuisine_191,Cuisine_192,Cuisine_193,Cuisine_194,Cuisine_195,Cuisine_196,Cuisine_197,Cuisine_198,Cuisine_199,Cuisine_200,Cuisine_201,Cuisine_202,Cuisine_203,Cuisine_204,Cuisine_205,Cuisine_206,Cuisine_207,Cuisine_208,Cuisine_209,Cuisine_210,Cuisine_211,Cuisine_212,Cuisine_213,Cuisine_214,Cuisine_215,Cuisine_216,Cuisine_217,Cuisine_218,Cuisine_219,Cuisine_220,Cuisine_221,Cuisine_222,Cuisine_223,Cuisine_224,Cuisine_225,Cuisine_226,Cuisine_227,Cuisine_228,Cuisine_229,Cuisine_230,Cuisine_231,Cuisine_232,Cuisine_233,Cuisine_234,Cuisine_235,Cuisine_236,Cuisine_237,Cuisine_238,Cuisine_239,Cuisine_240,Cuisine_241,Cuisine_242,Cuisine_243,Cuisine_244,Cuisine_245,Cuisine_246,Cuisine_247,Cuisine_248,Cuisine_249,Cuisine_250,Cuisine_251,Cuisine_252,Cuisine_253,Cuisine_254,Cuisine_255,Cuisine_256,Cuisine_257,Cuisine_258,Cuisine_259,Cuisine_260,Cuisine_261,Cuisine_262,Cuisine_263,Cuisine_264,Cuisine_265,Cuisine_266,Cuisine_267,Cuisine_268,Cuisine_269,Cuisine_270,Cuisine_271,Cuisine_272,Cuisine_273,Cuisine_274,Cuisine_275,Cuisine_276,Cuisine_277,Cuisine_278,Cuisine_279,Cuisine_280,Cuisine_281,Cuisine_282,Cuisine_283,Cuisine_284,Cuisine_285,Cuisine_286,Cuisine_287,Cuisine_288,Cuisine_289,Cuisine_290,Cuisine_291,Cuisine_292,Cuisine_293,Cuisine_294,Cuisine_295,Cuisine_296,Cuisine_297,Cuisine_298,Cuisine_299,Cuisine_300,Cuisine_301,Cuisine_302,Cuisine_303,Cuisine_304,Cuisine_305,Cuisine_306,Cuisine_307,Cuisine_308,Cuisine_309,Cuisine_310,Cuisine_311,Cuisine_312,Cuisine_313,Cuisine_314,Cuisine_315,Cuisine_316,Cuisine_317,Cuisine_318,Cuisine_319,Cuisine_320,Cuisine_321,Cuisine_322,Cuisine_323,Cuisine_324,Cuisine_325,Cuisine_326,Cuisine_327,Cuisine_328,Cuisine_329,Cuisine_330,Cuisine_331,Cui

In [23]:
cat_agg=['count','nunique']
num_agg=['min','mean','max','sum']
agg_col={'Average_Cost':num_agg, 'Minimum_Order':num_agg, 'Ratings':cat_agg,'Votes':num_agg}

for k in total_df.columns:
    if k.startswith('Cuisine_'):
        agg_col[k]=['count','nunique']
agg_col

{'Average_Cost': ['min', 'mean', 'max', 'sum'],
 'Minimum_Order': ['min', 'mean', 'max', 'sum'],
 'Ratings': ['count', 'nunique'],
 'Votes': ['min', 'mean', 'max', 'sum'],
 'Cuisine_0': ['count', 'nunique'],
 'Cuisine_1': ['count', 'nunique'],
 'Cuisine_2': ['count', 'nunique'],
 'Cuisine_3': ['count', 'nunique'],
 'Cuisine_4': ['count', 'nunique'],
 'Cuisine_5': ['count', 'nunique'],
 'Cuisine_6': ['count', 'nunique'],
 'Cuisine_7': ['count', 'nunique'],
 'Cuisine_8': ['count', 'nunique'],
 'Cuisine_9': ['count', 'nunique'],
 'Cuisine_10': ['count', 'nunique'],
 'Cuisine_11': ['count', 'nunique'],
 'Cuisine_12': ['count', 'nunique'],
 'Cuisine_13': ['count', 'nunique'],
 'Cuisine_14': ['count', 'nunique'],
 'Cuisine_15': ['count', 'nunique'],
 'Cuisine_16': ['count', 'nunique'],
 'Cuisine_17': ['count', 'nunique'],
 'Cuisine_18': ['count', 'nunique'],
 'Cuisine_19': ['count', 'nunique'],
 'Cuisine_20': ['count', 'nunique'],
 'Cuisine_21': ['count', 'nunique'],
 'Cuisine_22': ['count',

In [24]:
%time rest = total_df.groupby('Restaurant').agg(agg_col)

Wall time: 10.7 s


In [25]:
rest.shape

(8661, 2594)

In [26]:
rest['Rest_Count'] = total_df.groupby('Restaurant').size()
rest.shape

(8661, 2595)

In [27]:
total_df = total_df.merge(rest,on='Restaurant',how='left')
total_df.shape

C:\Users\vbanda3\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


(13868, 3897)

In [28]:
cols_to_drop = ['Restaurant','Delivery_Time','Cuisines','Rating','train_test']

test_df = total_df[total_df['train_test'] == 'test']
test_df = test_df.drop(cols_to_drop,axis=1)

train_df = total_df[total_df['train_test'] == 'train']
train_df['Delivery_Time'] = train_df['Delivery_Time'].astype('category')
y = train_df['Delivery_Time']

train_df = train_df.drop(cols_to_drop,axis=1)

X = train_df.copy()

test_df.shape, train_df.shape, y.dtype

((2774, 3892),
 (11094, 3892),
 CategoricalDtype(categories=[1, 2, 3, 4, 5, 6, 7], ordered=False))

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
X_train,X_val,y_train,y_val = train_test_split(train_df,y,test_size=0.3,random_state = 1994)

### RF

In [30]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score

In [31]:
rf = RandomForestClassifier(n_estimators=200,class_weight = 'balanced')
rf

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=200, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [32]:
%time rf.fit(X_train, y_train)

Wall time: 1min 2s


RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=200, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [33]:
dev_y_preds = rf.predict(X_train)
val_y_preds = rf.predict(X_val)
ts_pred = rf.predict(test_df)

In [34]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

print("Accuracy on train is:",accuracy_score(y_train,dev_y_preds))
print("Accuracy on test is:",accuracy_score(y_val,val_y_preds))

Accuracy on train is: 0.9979394719896973
Accuracy on test is: 0.7990387503754881


In [35]:
print("Confusion Matrix on Train" +"\n",confusion_matrix(y_train,dev_y_preds))
print("Confusion Matrix on Test" +"\n",confusion_matrix(y_val,val_y_preds))

Confusion Matrix on Train
 [[   4    0    0    0    0    0    0]
 [   0   46    0    0    0    0    0]
 [   0    0   13    0    0    0    0]
 [   0    1    1 5162    3    5    0]
 [   0    0    0    5 1856    0    0]
 [   0    0    0    1    0  656    0]
 [   0    0    0    0    0    0   12]]
Confusion Matrix on Test
 [[   8    0    5    2    1    0]
 [   0    0    7    0    0    0]
 [   0    0 2075  143   16    0]
 [   0    1  297  481   25    0]
 [   0    0   89   83   94    0]
 [   0    0    0    0    0    2]]


In [36]:
print("Classification report on Train"+"\n", classification_report(y_train,dev_y_preds))
print("Classification report on Train"+"\n", classification_report(y_val,val_y_preds))

Classification report on Train
               precision    recall  f1-score   support

           1       1.00      1.00      1.00         4
           2       0.98      1.00      0.99        46
           3       0.93      1.00      0.96        13
           4       1.00      1.00      1.00      5172
           5       1.00      1.00      1.00      1861
           6       0.99      1.00      1.00       657
           7       1.00      1.00      1.00        12

    accuracy                           1.00      7765
   macro avg       0.99      1.00      0.99      7765
weighted avg       1.00      1.00      1.00      7765

Classification report on Train
               precision    recall  f1-score   support

           2       1.00      0.50      0.67        16
           3       0.00      0.00      0.00         7
           4       0.84      0.93      0.88      2234
           5       0.68      0.60      0.64       804
           6       0.69      0.35      0.47       266
           7  

In [37]:
### Predicting on the Test 
sample['Delivery_Time'] = pd.Series(le.inverse_transform(ts_pred))
sample.head(10)

,Delivery_Time
0,45 minutes
1,30 minutes
2,30 minutes
3,30 minutes
4,30 minutes
5,30 minutes
6,30 minutes
7,30 minutes
8,30 minutes
9,30 minutes


In [38]:
sample['Delivery_Time'].value_counts()

30 minutes     2084
45 minutes      561
65 minutes      122
120 minutes       4
20 minutes        2
80 minutes        1
Name: Delivery_Time, dtype: int64

### RF Submission - 0.7919 Score

In [39]:
from datetime import datetime
_ = datetime.now().strftime('%Y%m%d%H%M%S')
sample.to_excel('overall_submission_rfpred'+'_'+str(_)+'.xlsx',index=False)

## LGBM

In [46]:
acc_tot=[]
y_pred_tot=[]

from sklearn.model_selection import KFold,StratifiedKFold

fold=StratifiedKFold(n_splits=10,shuffle=True,random_state=1994)

for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    m=LGBMClassifier(n_estimators=3000,random_state=199,learning_rate=0.03,colsample_bytree=0.2,objective='multiclass',class_weight = 'balanced')
    m.fit(X_train,y_train,eval_set=[(X_test, y_test)],eval_metric='logloss', early_stopping_rounds=100,verbose=200)
    preds=m.predict(X_test)
    print("acc: ",accuracy_score(preds,y_test))
    acc_tot.append(accuracy_score(preds,y_test))
    
    p = m.predict(test_df)
    y_pred_tot.append(p)

C:\Users\vbanda3\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Training until validation scores don't improve for 100 rounds.
[200]	valid_0's multi_logloss: 0.821165	valid_0's multi_logloss: 0.821165
[400]	valid_0's multi_logloss: 0.731661	valid_0's multi_logloss: 0.731661
[600]	valid_0's multi_logloss: 0.70125	valid_0's multi_logloss: 0.70125
[800]	valid_0's multi_logloss: 0.686562	valid_0's multi_logloss: 0.686562
[1000]	valid_0's multi_logloss: 0.678698	valid_0's multi_logloss: 0.678698
[1200]	valid_0's multi_logloss: 0.669182	valid_0's multi_logloss: 0.669182
[1400]	valid_0's multi_logloss: 0.660414	valid_0's multi_logloss: 0.660414
[1600]	valid_0's multi_logloss: 0.654076	valid_0's multi_logloss: 0.654076
[1800]	valid_0's multi_logloss: 0.650525	valid_0's multi_logloss: 0.650525
[2000]	valid_0's multi_logloss: 0.647454	valid_0's multi_logloss: 0.647454
[2200]	valid_0's multi_logloss: 0.646237	valid_0's multi_logloss: 0.646237
[2400]	valid_0's multi_logloss: 0.644681	valid_0's multi_logloss: 0.644681
Early stopping, best iteration is:
[2473]	v

[2600]	valid_0's multi_logloss: 0.623096	valid_0's multi_logloss: 0.623096
Early stopping, best iteration is:
[2589]	valid_0's multi_logloss: 0.623004	valid_0's multi_logloss: 0.623004
acc:  0.796028880866426
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's multi_logloss: 0.827577	valid_0's multi_logloss: 0.827577
[400]	valid_0's multi_logloss: 0.729654	valid_0's multi_logloss: 0.729654
[600]	valid_0's multi_logloss: 0.690522	valid_0's multi_logloss: 0.690522
[800]	valid_0's multi_logloss: 0.670207	valid_0's multi_logloss: 0.670207
[1000]	valid_0's multi_logloss: 0.656292	valid_0's multi_logloss: 0.656292
[1200]	valid_0's multi_logloss: 0.646019	valid_0's multi_logloss: 0.646019
[1400]	valid_0's multi_logloss: 0.637872	valid_0's multi_logloss: 0.637872
[1600]	valid_0's multi_logloss: 0.630338	valid_0's multi_logloss: 0.630338
[1800]	valid_0's multi_logloss: 0.624832	valid_0's multi_logloss: 0.624832
[2000]	valid_0's multi_logloss: 0.621225	valid_0's multi_

In [47]:
np.mean(acc_tot)

0.7832023151042727

In [48]:
np.mean(y_pred_tot,0)

array([4.2, 4. , 4.4, ..., 4.7, 5.3, 6. ])

In [63]:
y_pred_tot_df = pd.DataFrame(y_pred_tot)
y_pred_tot_df.shape

(10, 2774)

In [64]:
y_pred_tot_df = y_pred_tot_df.T

In [65]:
y_pred_tot_df['Mean'] = np.mean(y_pred_tot,0)

In [66]:
y_pred_tot_df.head()

,0,1,2,3,4,5,6,7,8,9,Mean
0,4,4,4,4,4,5,4,4,4,5,4.2
1,4,4,4,4,4,4,4,4,4,4,4.0
2,6,4,4,6,4,4,4,4,4,4,4.4
3,4,4,4,4,4,4,4,4,4,4,4.0
4,5,4,4,5,4,5,4,4,4,5,4.4


In [67]:
y_pred_tot_df['Mean'].value_counts()

4.0    1570
5.0     464
6.0      91
4.9      86
4.1      79
5.9      54
4.2      54
5.1      46
4.8      39
4.6      35
5.8      35
4.3      32
5.2      29
4.4      28
4.7      23
5.6      20
5.3      20
4.5      19
5.4      17
5.5      11
5.7      11
2.0       3
3.5       2
2.3       2
3.2       2
7.0       1
3.0       1
Name: Mean, dtype: int64

In [68]:
y_pred_tot_df['Actual'] = 0

y_pred_tot_df['Actual'] = np.round(y_pred_tot_df['Mean'],0)
y_pred_tot_df['Actual'].value_counts()

4.0    1784
5.0     759
6.0     222
2.0       5
3.0       3
7.0       1
Name: Actual, dtype: int64

In [69]:
### Predicting on the Test 
y_pred_tot_df['Actual'] = y_pred_tot_df['Actual'].astype('int')
sample['Delivery_Time'] = pd.Series(le.inverse_transform(y_pred_tot_df['Actual']))
sample.head(10)

,Delivery_Time
0,30 minutes
1,30 minutes
2,30 minutes
3,30 minutes
4,30 minutes
5,30 minutes
6,45 minutes
7,30 minutes
8,30 minutes
9,30 minutes


### LGBM Submission - 0.8215573 Score 

In [70]:
from datetime import datetime
_ = datetime.now().strftime('%Y%m%d%H%M%S')
sample.to_excel('overall2_submission_lgbmpred'+'_'+str(_)+'.xlsx',index=False)